In [1]:
import pandas as pd

df = pd.read_csv("../data/" + 'train_data_criteria.csv')

In [3]:
import ast

def parse_q_a_criteria(q_a_criteria):
    """
    Converts a string of question-answer pairs into a list of formatted strings.

    Parameters:
    q_a_criteria (str): Input string containing question-answer pairs.

    Returns:
    list: List of strings combining questions and answers.
    """
    result = []
    # Split by newline to handle individual JSON-like entries
    for entry in q_a_criteria.split('\n'):
        try:
            # Safely evaluate the string to a dictionary
            qa_dict = ast.literal_eval(entry)
            if 'Question' in qa_dict and 'Answer' in qa_dict:
                # Format the question-answer pair
                result.append(f"{qa_dict['Question']} {qa_dict['Answer']}")
        except (ValueError, SyntaxError):
            continue
    return result

criteria

In [6]:
#look for trials where q_a_criteria do not have any '{' character, replace text with {'Question': 'No question', 'Answer': 'No answer'}

df['q_a_criteria'] = df['q_a_criteria'].apply(lambda x: x if '{' in x else "{'Question': 'No question', 'Answer': 'No answer'}")

#look for trials where q_a_criteria have any '{' character

df['q_a_criteria'].apply(lambda x: '{' in x).value_counts()

df_prime = df[df['q_a_criteria'].str.endswith('}') == False]

In [7]:
#remove the sibstring after the last '}' 

df_prime['q_a_criteria'] = df_prime['q_a_criteria'].apply(lambda x: x[:x.rfind('}')+1])

/tmp/ipykernel_4019616/755035883.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prime['q_a_criteria'] = df_prime['q_a_criteria'].apply(lambda x: x[:x.rfind('}')+1])


In [8]:
#remove the rows that are in df_prime from df. then concatenate df and df_prime

df = df[~df['q_a_criteria'].str.endswith('}') == False]

df = pd.concat([df, df_prime])


(40000, 10)

In [9]:
#reset index

df.reset_index(drop=True, inplace=True)

In [10]:
df['q_a_criteria'] = df['q_a_criteria'].apply(parse_q_a_criteria)

In [13]:
df.to_pickle("../data/" + 'train_data_criteria.pkl')